In [ ]:
pip install understat

In [1]:
import asyncio
import json
import aiohttp
from understat import Understat
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
#Collecting the data for each team in that particular season that the player has played against. This will be the training data

async with aiohttp.ClientSession() as session:
    understat = Understat(session)
    matches = await understat.get_player_matches(1250) #enter the understat id of the player. Here we have taken Mohamed Salah.
    league_stats_2021= await understat.get_league_table("EPL", "2021")
    league_stats_2021_df=pd.DataFrame(league_stats_2021[1:], columns=league_stats_2021[0])
    league_stats_2020= await understat.get_league_table("EPL", "2020")
    league_stats_2020_df=pd.DataFrame(league_stats_2020[1:], columns=league_stats_2020[0])
    league_stats_2019= await understat.get_league_table("EPL", "2019")
    league_stats_2019_df=pd.DataFrame(league_stats_2019[1:], columns=league_stats_2019[0])
    league_stats_2018= await understat.get_league_table("EPL", "2018")
    league_stats_2018_df=pd.DataFrame(league_stats_2018[1:], columns=league_stats_2018[0])
    team_df=pd.DataFrame(columns=['SpSlow', 'SpFast', 'xG', 'xGA', 'PPDA', 'ODC', 'home', 'npxG', 'xA'])
    print("hi")
    i=-1
    for match in matches:
        home=1
        if(match['season']!='2020'):
            if(match['h_team']=='Liverpool'):
                home=2
            else:
                home=0
        i=i+1
        if((match['h_team']!='Liverpool') & (match['a_team']!='Liverpool')):
            break
        if(match['season']=='2016'):
            break
        target_team= match['h_team'] if match['h_team'] != 'Liverpool' else match['a_team']
        team_stats = await understat.get_team_stats(target_team, match['season'])
        league_stats=pd.Series()
        if(match['season']=='2021'):
            league_stats=league_stats_2021_df[league_stats_2021_df['Team']==target_team]        
        if(match['season']=='2020'):
            league_stats=league_stats_2020_df[league_stats_2020_df['Team']==target_team]
        if(match['season']=='2019'):
            league_stats=league_stats_2019_df[league_stats_2019_df['Team']==target_team]
        if(match['season']=='2018'):
            league_stats=league_stats_2018_df[league_stats_2018_df['Team']==target_team]            
#         print(league_stats)
        row=[(team_stats['attackSpeed']['Slow']['against']['xG'])/league_stats.iloc[0]['xGA'], (team_stats['attackSpeed']['Fast']['against']['xG'])/league_stats.iloc[0]['xGA'], league_stats.iloc[0]['xG'],league_stats.iloc[0]['xGA'],league_stats.iloc[0]['PPDA'],league_stats.iloc[0]['ODC'], home, (float(match['npxG'])/float(match['time']))*90, (float(match['xA'])/float(match['time']))*90]
        team_df.loc[i]=row

In [16]:
#Training the model for the player


team_df = team_df.astype(float)
X_train=team_df.drop(columns=['npxG', 'xA'])
y_train=team_df['npxG']
y_train_assist=team_df['xA']
clf=RandomForestRegressor()
clf.fit(X_train, y_train)
clfassist=RandomForestRegressor()
clfassist.fit(X_train, y_train_assist)
index=0

In [ ]:
#Cross-validating the models

scores = cross_val_score(clf, X_train, y_train, cv=10)
print(scores)
scores_assist = cross_val_score(clfassist, X_train, y_train_assist, cv=10)
print(scores_assist)

In [ ]:
#Collecting the data for each team the prediction is to be made for

async with aiohttp.ClientSession() as session2:
    understat2 = Understat(session2)
    league_team_stats= await understat2.get_league_table("EPL", "2021")
    league_team_stats_df=pd.DataFrame(league_team_stats[1:], columns=league_team_stats[0])
    result_df=pd.DataFrame(columns=['Team', 'npxG', 'xA'])
    for i,team in league_team_stats_df.iterrows():
        predict_team_stats = await understat2.get_team_stats(team[0], '2021')
        predict_league_stats=pd.Series()
        predict_league_stats=league_stats_2021_df[league_stats_2021_df['Team']==team[0]]
        predictrowhome_df=pd.DataFrame(columns=['SpSlow', 'SpFast', 'xG', 'xGA', 'PPDA', 'ODC', 'home'])
        predictrowaway_df=pd.DataFrame(columns=['SpSlow', 'SpFast', 'xG', 'xGA', 'PPDA', 'ODC', 'home'])
        predictrowhome=[(predict_team_stats['attackSpeed']['Slow']['against']['xG'])/predict_league_stats.iloc[0]['xGA'], (predict_team_stats['attackSpeed']['Fast']['against']['xG'])/predict_league_stats.iloc[0]['xGA'], predict_league_stats.iloc[0]['xG'],predict_league_stats.iloc[0]['xGA'],predict_league_stats.iloc[0]['PPDA'],predict_league_stats.iloc[0]['ODC'], 2]
        predictrowaway=[(predict_team_stats['attackSpeed']['Slow']['against']['xG'])/predict_league_stats.iloc[0]['xGA'], (predict_team_stats['attackSpeed']['Fast']['against']['xG'])/predict_league_stats.iloc[0]['xGA'], predict_league_stats.iloc[0]['xG'],predict_league_stats.iloc[0]['xGA'],predict_league_stats.iloc[0]['PPDA'],predict_league_stats.iloc[0]['ODC'], 0]
        predictrowhome_df.loc[0]=predictrowhome
        predictrowaway_df.loc[0]=predictrowaway
        predictrowhome_df = predictrowhome_df.astype(float)
        predictrowaway_df = predictrowaway_df.astype(float)
        rowhome=[team[0]+' Home',clf.predict(predictrowhome_df)[0],clfassist.predict(predictrowhome_df)[0]]
        rowaway=[team[0]+' Away',clf.predict(predictrowaway_df)[0],clfassist.predict(predictrowaway_df)[0]]
        result_df.loc[index]=rowhome
        result_df.loc[index+1]=rowaway
        index=index+2
    result_df['npxG+xA']=(result_df['npxG']+result_df['xA'])

In [20]:
#Predicting the npXG and xA against different teams based on the current season's stats.

result_df.sort_values("npxG+xA", ascending=False)

,Team,npxG,xA,npxG+xA
35,Leicester Away,1.156491,0.449104,1.605595
34,Leicester Home,0.998842,0.531470,1.530312
23,Manchester United Away,1.129161,0.322716,1.451877
27,Manchester City Away,1.098561,0.339428,1.437989
41,Aston Villa Away,1.111465,0.323859,1.435323
40,Aston Villa Home,0.928742,0.493536,1.422278
26,Manchester City Home,0.916925,0.493219,1.410144
51,Leeds Away,0.987106,0.407662,1.394768
22,Manchester United Home,0.912672,0.453259,1.365931
21,Liverpool Away,1.081512,0.282090,1.363602
